# Bayesian Transfer Learning for Deep Networks

In this project we are concerned with **Bayesian Deep Learning**. Specifically, we want to know whether having a deep Bayesian model will improve the transfer of learning. Our hypothesis is that that knowledge gained from training a model on tasks **A** and then using the learned weights as a basis for learning on tasks $B$ will perform better than training **B** from scratch - assuming the domains are similar.

![Transfer Learning](https://image.slidesharecdn.com/13aibigdata-160606103446/95/aibigdata-lab-2016-transfer-learning-7-638.jpg?cb=1465209397)

We use Bayes By Backprop introduced by [Blundell, 2015](https://arxiv.org/abs/1505.05424). to learn a probability distribution over each of the weights in the network. These weight distributions are fitted using variational inference given some prior.

By inferring the posterior weight distribution in task **A** $p(w|D_A)$, a model is trained which is able to solve the second task **B** when exposed to new data $D_B$, while remembering task **A**. Variational Bayasian approximations of $p(w|D_A)$ are considered for this operation.

> The model constructed in this notebook tries to dynamically adapt its weights when confronted with new tasks. A method named **elastic weight consolidation (EWC)** ([Kirkpatrick, 2016](http://www.pnas.org/content/114/13/3521.full.pdf)) is implemented that considers data from two different tasks as independent.

### Import packages

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.optim as optim
import torch.nn.functional as F

from collections import Counter, defaultdict, OrderedDict
from tqdm import tqdm
import math
import os, pickle, gc
import seaborn as sns
from scipy.stats import norm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

### Defining $\beta$ experiments

### Defining Normalizing Flows experiments

In [2]:

class NormalizingFlows(nn.Module):
    def __init__(self, n, features):
        super(NormalizingFlows, self).__init__()
        flows = [PlanarNormalizingFlow(features) for _ in range(n)]
        self.flows = nn.ModuleList(flows)

    def forward(self, z):
        log_dets = []

        for flow in self.flows:
            z, log_det_jacobian = flow(z)
            log_dets.append(log_det_jacobian)

        return z, log_dets


class PlanarNormalizingFlow(nn.Module):
    """
    Based on Normalizing Flow implementation from Parmesan
    https://github.com/casperkaae/parmesan
    """
    def __init__(self, features):
        super(PlanarNormalizingFlow, self).__init__()
        self.u = Parameter(torch.randn(features))
        self.w = Parameter(torch.randn(features))
        self.b = Parameter(torch.ones(1))

    def forward(self, z):
        # Create uhat such that it is parallel to w
        uw = torch.dot(self.u, self.w)
        muw = -1 + F.softplus(uw)
        uhat = self.u + (muw - uw) * torch.transpose(self.w, 0, -1) / torch.sum(self.w ** 2)

        # Equation 21 - Transform z
        zwb = torch.mv(z, self.w) + self.b

        f_z = z + (uhat.view(1, -1) * F.tanh(zwb).view(-1, 1))

        # Compute the Jacobian using the fact that
        # tanh(x) dx = 1 - tanh(x)**2
        psi = (1 - F.tanh(zwb)**2).view(-1, 1) * self.w.view(1, -1)
        psi_u = torch.mv(psi, uhat)

        # Return the transformed output along
        # with log determninant of J
        logdet_jacobian = torch.log(torch.abs(1 + psi_u) + 1e-8)

        return f_z, logdet_jacobian

### Defining transfer learning experiments

### (header missing)

In [3]:
def log_sum_exp(A, dim=-1, keepdim=False, sum_op=torch.sum):
    """Computes `log(exp(A).sum(axis=axis))` avoiding numerical issues using the log-sum-exp trick.
    Direct calculation of :math:`\log \sum_i \exp A_i` can result in underflow or overflow numerical
    issues. Big positive values can cause overflow :math:`\exp A_i = \inf`, and big negative values
    can cause underflow :math:`\exp A_i = 0`. The latter can eventually cause the sum to go to zero
    and finally resulting in :math:`\log 0 = -\inf`.
    The log-sum-exp trick avoids these issues by using the identity,
    .. math::
        \log \sum_i \exp A_i = \log \sum_i \exp(A_i - c) + c, \text{using},  \\
        c = \max A.7
    This avoids overflow, and while underflow can still happen for individual elements it avoids
    the sum being zero.

    Parameters
    ----------
    A : tensor
        Tensor of which we wish to compute the log-sum-exp.
    axis : int, tuple, list, None
        Axis or axes to sum over; None (default) sums over all axes.
    sum_op : function
        Summing function to apply; default is torch.sum, but can also be torch.mean for log-mean-exp.

    Returns
    -------
    tensor
        The log-sum-exp of `A`, dimensions over which is summed will be dropped.
    """
    A_max = torch.max(A,dim=dim,keepdim=True)[0]
    B = torch.log(sum_op(torch.exp(A - A_max), dim=dim, keepdim=True)) + A_max
    
    if not keepdim:
        B = B.squeeze(dim)
    
    return B



class Distribution():
    def pdf(self, x):
        raise NotImplementedError

    def logpdf(self, x):
        raise NotImplementedError

    def cdf(self, x):
        raise NotImplementedError

    def logcdf(self, x):
        raise NotImplementedError

    def sample(self):
        raise NotImplementedError

    
class Normal(Distribution):
    # scalar version
    def __init__(self, loc, logvar):
        self.loc = loc
        self.logvar = logvar
        self.shp = loc.size()

        super(Normal,self).__init__()

    def logpdf(self, x, eps=0.0):
        c = - float(0.5 * math.log(2 * math.pi))
        return c - 0.5*self.logvar - (x - self.loc).pow(2) / (2 * torch.exp((self.logvar)) + eps)

    def pdf(self, x):
        return torch.exp(self.logpdf(x))

    def sample(self):
        if self.loc.is_cuda:
            eps = torch.cuda.FloatTensor(self.shp).normal_()
        else:
            eps = torch.FloatTensor(self.shp).normal_()
        return self.loc + torch.exp(0.5*self.logvar) * Variable(eps)
    
    def entropy(self):
        return 0.5 * math.log(2. * math.pi * math.e) + 0.5*self.logvar



def kl_normal2_normal2(mean1, log_var1, mean2, log_var2, eps=0.0):
    """
    Compute closed-form solution to the KL-divergence between two Gaussians parameterized
    with diagonal log variance.
    .. math::
       D_{KL}[q||p] &= -\int p(x) \log q(x) dx + \int p(x) \log p(x) dx     \\
                    &= -\int \mathcal{N}(x; \mu_2, \sigma^2_2) \log \mathcal{N}(x; \mu_1, \sigma^2_1) dx
                        + \int \mathcal{N}(x; \mu_2, \sigma^2_2) \log \mathcal{N}(x; \mu_2, \sigma^2_2) dx     \\
                    &= \frac{1}{2} \log(2\pi\sigma^2_2) + \frac{\sigma^2_1 + (\mu_1 - \mu_2)^2}{2\sigma^2_2}
                        - \frac{1}{2}( 1 + \log(2\pi\sigma^2_1) )      \\
                    &= \log \frac{\sigma_2}{\sigma_1} + \frac{\sigma^2_1 + (\mu_1 - \mu_2)^2}{2\sigma^2_2} - \frac{1}{2}
    """
    return 0.5*log_var2 - 0.5*log_var1 + (torch.exp(log_var1) + (mean1 - mean2)**2) / (2*torch.exp(log_var2) + eps) - 0.5

    
class FixedMixtureNormal(nn.Module):   #needs to be a nn.Moudle to register the parameters correcly
    # takes loc, logvar and pi as list of float values and assumes they are shared across all dimenisions
    def __init__(self, loc, logvar, pi):
        super(FixedMixtureNormal,self).__init__()
        assert sum(pi) -1 < 0.0001
        self.loc = Parameter(torch.from_numpy(np.array(loc)).float(),requires_grad=False)
        self.logvar = Parameter(torch.from_numpy(np.array(logvar)).float(),requires_grad=False)
        self.pi = Parameter(torch.from_numpy(np.array(pi)).float(),requires_grad=False)

    def _component_logpdf(self, x, eps=0.0):
        ndim = len(x.size())
        shpexpand = ndim*(None,)
        x = x.unsqueeze(-1)
        
        c = - float(0.5 * math.log(2 * math.pi))
        loc = self.loc[shpexpand]
        logvar = self.logvar[shpexpand]
        pi = self.pi[shpexpand]
        
        return c - 0.5*logvar - (x - loc).pow(2) / (2 * torch.exp(logvar) + eps)

    def logpdf(self,x):
        ndim = len(x.size())
        shpexpand = ndim*(None,)
        pi = self.pi[shpexpand]
        px = torch.exp(self._component_logpdf(x))   #... x num_components
        return torch.log(torch.sum(pi*px,-1))          

    
    
class FixedNormal(Distribution):
    # takes loc and logvar as float values and assumes they are shared across all dimenisions
    def __init__(self, loc, logvar):
        self.loc = loc
        self.logvar = logvar
        super(FixedNormal,self).__init__()

    def logpdf(self, x, eps=0.0):
        c = - float(0.5 * math.log(2 * math.pi))
        return c - 0.5*self.logvar - (x - self.loc).pow(2) / (2 * math.exp((self.logvar)) + eps)

    

### Defining prior weight distributions, activation functions, and loss function

correct? is that all? Better to make multiple cells out of it?

In [4]:
def distribution_selector(loc, logvar, pi):    
    if isinstance(logvar,(list,tuple)) and isinstance(pi,(list,tuple)):
        assert len(logvar) == len(pi)
        num_components = len(logvar)
        print('Mixture of Normal prior, nc:',num_components)
        if not isinstance(loc,(list,tuple)):
            loc = (loc,)*num_components
        return FixedMixtureNormal(loc,logvar,pi)
    else:
        print('Normal prior')
        return FixedNormal(loc,logvar)


class BBBLinearFactorial(nn.Module):
    def __init__(self,in_features, out_features, p_logvar_init=-3, p_pi=1.0, q_logvar_init=-5):
        # p_logvar_init, p_pi can be either
        #    (list/tuples): prior model is a mixture of gaussians components=len(p_pi)=len(p_logvar_init)
        #    float: Gussian distribution
        # q_logvar_init: float, the approximate posterior is currently always a factorized gaussian
        super(BBBLinearFactorial,self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.p_logvar_init = p_logvar_init
        self.q_logvar_init = q_logvar_init
                
        #Approximate Posterior model
        self.qw_mean = Parameter(torch.Tensor(out_features, in_features))
        self.qw_logvar = Parameter(torch.Tensor(out_features, in_features))
        self.qb_mean = Parameter(torch.Tensor(out_features))
        self.qb_logvar = Parameter(torch.Tensor(out_features))   
        
        self.normalizing_flow_w = NormalizingFlows(n=5, features=in_features*out_features)
        self.normalizing_flow_b = NormalizingFlows(n=5, features=out_features)

        self.qw = Normal(loc=self.qw_mean, logvar=self.qw_logvar)
        self.qb = Normal(loc=self.qb_mean, logvar=self.qb_logvar)
            
        #Prior Model (the prior model does not have any trainable parameters so we use special versions of the normal distributions)
        self.pw = distribution_selector(loc=0.0, logvar=p_logvar_init, pi=p_pi)
        self.pb = distribution_selector(loc=0.0, logvar=p_logvar_init, pi=p_pi)

        #initialize all paramaters
        self.reset_parameters()

    def reset_parameters(self):
        #initialize (learnable) approximate posterior parameters        
        stdv = 10. / math.sqrt(self.in_features)
        self.qw_mean.data.uniform_(-stdv, stdv)
        self.qw_logvar.data.uniform_(-stdv, stdv).add_(self.q_logvar_init)
        self.qb_mean.data.uniform_(-stdv, stdv)
        self.qb_logvar.data.uniform_(-stdv, stdv).add_(self.q_logvar_init)

        
    def forward(self, input):
        raise NotImplementedError()
        
    def probforward(self, input, MAP=False):
        #input: BS, in_features
        #W: BS, in_features
        #MAP: maximum a posterior (return the mean instead of sampling from the distributions)
        if MAP:
            w_sample = self.qw.loc
            b_sample = self.qb.loc
        else:
            w_sample = self.qw.sample()
            b_sample = self.qb.sample()

        f_w_sample, log_det_w = self.normalizing_flow_w(w_sample.view(1, -1))
        f_b_sample, log_det_b = self.normalizing_flow_b(b_sample.view(1, -1))

        f_w_sample = f_w_sample.view(w_sample.size())
        f_b_sample = f_b_sample.view(b_sample.size())

        # Subtracting log det J is the same as multiplying by 1/(det J)
        qw_logpdf = self.qw.logpdf(w_sample) - sum(log_det_w)
        qb_logpdf = self.qb.logpdf(b_sample) - sum(log_det_b)

        kl_w = torch.sum(qw_logpdf - self.pw.logpdf(f_w_sample))
        kl_b = torch.sum(qb_logpdf - self.pb.logpdf(f_b_sample))
        kl = kl_w + kl_b
        
        diagnostics = {'kl_w':kl_w.data.mean(), 'kl_b':kl_b.data.mean(), 
                       'Hq_w': self.qw.entropy().data.mean(), 'Hq_b': self.qb.entropy().data.mean()}   #Hq_w and Hq_b are the differential entropy
        output = F.linear(input,f_w_sample, f_b_sample)

        return output, kl, diagnostics


    def __repr__(self):
        return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' \
            + str(self.out_features) + ')'
    
    
    
class BBBMLP(nn.Module):
    def __init__(self,in_features, num_class, num_hidden, num_layers, p_logvar_init=-3, p_pi=1.0, q_logvar_init=-5):
        #create a simple MLP model with probabilistic weights
        super(BBBMLP, self).__init__()        
        layers = [BBBLinearFactorial(in_features=in_features,out_features=num_hidden, p_logvar_init=p_logvar_init, p_pi=p_pi, q_logvar_init=q_logvar_init), nn.ReLU()]
        for i in range(num_layers-1):
            layers += [BBBLinearFactorial(in_features=num_hidden,out_features=num_hidden, p_logvar_init=p_logvar_init, p_pi=p_pi, q_logvar_init=q_logvar_init), nn.ReLU()]
        layers += [BBBLinearFactorial(in_features=num_hidden,out_features=num_class, p_logvar_init=p_logvar_init, p_pi=p_pi, q_logvar_init=q_logvar_init)]
        
        self.layers = nn.ModuleList(layers)
        self.loss = nn.CrossEntropyLoss()
        
    def probforward(self,x, MAP=False):
        diagnostics = defaultdict(list)
        kl = 0
        for l in self.layers:
            if hasattr(l, 'probforward' ) and callable( l.probforward ): 
                x, _kl, _diagnostics = l.probforward(x,MAP=MAP) 
                kl += _kl
                for k,v in _diagnostics.items():
                    diagnostics[k].append(v)
            else:
                x = l(x)        
        logits = x
        return logits, kl, diagnostics

    def load_prior(self, state_dict):
        d_q = {k: v for k, v in state_dict.items() if "q" in k}
        for i, layer in enumerate(self.layers):
            if type(layer) is BBBLinearFactorial:
                layer.pw = Normal(loc=Variable(d_q["layers.{}.qw_mean".format(i)]).cuda(),
                                  logvar=Variable(d_q["layers.{}.qw_logvar".format(i)]).cuda())

                layer.pb = Normal(loc=Variable(d_q["layers.{}.qb_mean".format(i)]).cuda(),
                                  logvar=Variable(d_q["layers.{}.qb_logvar".format(i)]).cuda())


    def getloss(self,x,y, beta, MAP=False):
        logits, kl, _diagnostics = self.probforward(x, MAP=MAP)
        #_diagnostics is here a dictinary of list of floats
        # We need the dataset_size in order to 'spread' the KL divergence across all samples - this is dscribed in EQ (8) in Blundel et. al. 2015

        logpy = -self.loss(logits,y) #sample average

        ll = logpy - beta*kl   #ELBO
        loss = -ll

        acc = (logits.max(1)[1].data == th_y).float().mean() #accuracy
        
        #the xxx.data.mean() is just an easy way to transfer to cpu and convert from torch to normal floats
        diagnostics = {'loss': [loss.data.mean()],
                       'll': [ll.data.mean()],
                       'kl': [kl.data.mean()],
                       'logpy': [logpy.data.mean()],
                       'acc': [acc],
                       'kl_w': _diagnostics['kl_w'],
                       'kl_b': _diagnostics['kl_b'],
                       'Hq_w': _diagnostics['Hq_w'],
                       'Hq_b': _diagnostics['Hq_b'],}
        return logits, loss, diagnostics
    
    

def plothist(model,filename):
    N = norm(loc=0,scale=np.exp(p_logvar_init))
    x =np.linspace(-0.5,0.5,100)
    W = torch.cat([model.layers[0].qw_mean.view(-1), model.layers[2].qw_mean.view(-1), model.layers[4].qw_mean.view(-1)]).data.cpu().numpy()
    b = torch.cat([model.layers[0].qb_mean.view(-1), model.layers[2].qb_mean.view(-1), model.layers[4].qb_mean.view(-1)]).data.cpu().numpy()

    plt.figure(figsize=(10,5))
    plt.subplot(121)
    _ = plt.hist(W,np.linspace(-0.5,0.5,100),normed=True,label='q samples')
    plt.plot(x, N.pdf(x),label='prior pdf')
    plt.xlim([-0.5,0.5])
    plt.ylim([0,10])
    plt.legend()
    plt.title('Weights')

    plt.subplot(122)
    _ = plt.hist(b,np.linspace(-0.5,0.5,100),normed=True,label='q samples')
    plt.plot(x, N.pdf(x),label='prior pdf')
    plt.xlim([-0.5,0.5])
    plt.ylim([0,10])
    plt.legend()
    plt.title('bias')
    
    plt.savefig(filename)
    plt.close('all')

def addres(old,new):
    for k in new.keys():
        if k in old:
            old[k] = [vo+vn for vo,vn in zip(old[k],new[k])]
        else:
            old[k] = new[k]
    return old
    
def averres(res,num_batch):
    for k in res.keys():
        res[k] = [v/num_batch for v in res[k]]
    return res
    
def listdict2dictlist(LD):
    DL = dict(zip(sorted(LD[0].keys()),zip(*[[v for k,v in sorted(d.items())]for d in LD])))
    return DL
    

### Loading data

In [5]:
from operator import __or__
from functools import reduce
data = np.load('mnist.npz')  #Download from https://www.dropbox.com/s/k92825vinroxh6i/mnist.npz?dl=0
xtrain, ytrain = data['x_train'],data['y_train']
xvalid, yvalid = data['x_valid'],data['y_valid']
xtest, ytest = data['x_test'],data['y_test']
    
    


#MU, SIGMA = xtrain.mean(keepdims=True), xtrain.std(keepdims=True)
#xtrain = (xtrain-0.5)/0.5
#xvalid = (xvalid-0.5)/0.5
#xtest = (xtest-0.5)/0.5


### Hyperparameters and experiment settings

In [ ]:
batch_size = 128
nsamples = 10
dataset_size, in_features = xtrain.shape
num_hidden = 400
num_layers = 2
p_logvar_init = 0#(-1,-2,-3,-4,-5)
q_logvar_init = -4
p_pi = 1.0 #(1./10.,2./10.,4./10.,2./10.,1./10.)

num_class = 10
learningrate = 1e-4
num_epochs = 50
cuda = torch.cuda.is_available()
save_every_epoch = 10


expname = 'norm_flow'
weights_dir = os.path.join(expname,'weights')
figure_dir = os.path.join(expname,'figures')

logfile = os.path.join(expname,'logfile.txt')
diagnosticsfile = os.path.join(expname,'diagnostics.pkl')
weightsfile = os.path.join(weights_dir,'model_epoch%i.pkl')
histfigurefile = os.path.join(figure_dir,'weighthistogram_epoch%i.png')


if not os.path.exists(expname): os.makedirs(expname)
if not os.path.exists(weights_dir): os.makedirs(weights_dir)  
if not os.path.exists(figure_dir): os.makedirs(figure_dir)  
with open(logfile,'w') as fh: fh.write('')


### Training the model

In [6]:
model = BBBMLP(in_features=in_features, num_class=num_class, num_hidden=num_hidden, num_layers=num_layers, p_logvar_init=p_logvar_init, p_pi=p_pi, q_logvar_init=q_logvar_init)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learningrate)
th_x = torch.FloatTensor(batch_size*nsamples,in_features).fill_(0)
th_y = torch.LongTensor(batch_size*nsamples).fill_(0)

#d = pickle.load(open("domain_a_{}/weights/model_epoch99.pkl".format(f), "rb"))
#model.load_prior(d)

if cuda:
    model.cuda()
    th_x = th_x.cuda()
    th_y = th_y.cuda()


def run_epoch(np_x, np_y, batch_size, epoch, MAP=False, is_training=False):
    N = np_x.shape[0]
    idx = np.arange(N)
    np.random.shuffle(idx)
    diagnostics = {}
    nbatch_per_epoch = N//batch_size

    for i in tqdm(range(nbatch_per_epoch)):
        batch_idx = idx[i*batch_size : (i+1)*batch_size]

        th_x.copy_(torch.from_numpy(np.tile(np_x[batch_idx],[nsamples,1])))
        th_y.copy_(torch.from_numpy(np.tile(np_y[batch_idx],[nsamples])))

        beta = 2 ** (nbatch_per_epoch - (epoch + 1)) / (2 ** nbatch_per_epoch - 1)
        logits, loss, _diagnostics = model.getloss(Variable(th_x),Variable(th_y), beta=beta, MAP=MAP)
        diagnostics = addres(diagnostics, _diagnostics)
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    diagnostics = averres(diagnostics,nbatch_per_epoch)
    return diagnostics



diagnostics_batch_train, diagnostics_batch_valid, diagnostics_batch_valid_MAP = [],[],[]
for e in range(num_epochs):

    if e % save_every_epoch == 0:
        #get weights before training
        weights = model.state_dict()
        for k in weights: 
            weights[k] = weights[k].cpu()
        plothist(model,histfigurefile%(e))   

    diagnostics_batch_train += [run_epoch(xtrain, ytrain, batch_size, e, MAP=False, is_training=True)]
    diagnostics_batch_valid += [run_epoch(xvalid, yvalid, batch_size, e, MAP=False, is_training=False)]
    diagnostics_batch_valid_MAP += [run_epoch(xvalid, yvalid, batch_size, e, MAP=True, is_training=False)]

    ltr = "Train %i | "%(e) + ", ".join(["%s: %s"%(k,"|".join(["%0.3f"%(_v) for _v in v])) for k,v in sorted(diagnostics_batch_train[-1].items())])
    lte = "Test %i | "%(e)     + ", ".join(["%s: %s"%(k,"|".join(["%0.3f"%(_v) for _v in v])) for k,v in sorted(diagnostics_batch_valid[-1].items())])
    ltemap = "MAP-Test %i | "%(e)     + ", ".join(["%s: %s"%(k,"|".join(["%0.3f"%(_v) for _v in v])) for k,v in sorted(diagnostics_batch_valid_MAP[-1].items())])

    print('\n'.join([ltr,lte,ltemap]))
    with open(logfile,'a') as fh: fh.write('\n'.join([ltr,lte,ltemap]) + '\n')

    if e % save_every_epoch == 0:
        with open(diagnosticsfile,'wb') as fh: 
            pickle.dump({'train':listdict2dictlist(diagnostics_batch_train),
                         'valid':listdict2dictlist(diagnostics_batch_valid),
                         'validMAP':listdict2dictlist(diagnostics_batch_valid_MAP)},fh)
        with open(weightsfile%(e),'wb') as fh: pickle.dump(weights,fh)
    gc.collect()
          
with open(weightsfile%(e),'wb') as fh: pickle.dump(weights,fh)

Normal prior
Normal prior
Normal prior
Normal prior
Normal prior
Normal prior


100%|██████████| 78/78 [00:06<00:00, 11.86it/s]


Train 0 | Hq_b: -0.589|-0.582|-0.562, Hq_w: -0.581|-0.576|-0.582, acc: 0.134, kl: 1911262.741, kl_b: 1657.242|1409.687|50.421, kl_w: 1249785.139|642001.854|16358.391, ll: -975944.013, logpy: -20312.643, loss: 975944.013
Test 0 | Hq_b: -0.588|-0.582|-0.559, Hq_w: -0.581|-0.574|-0.582, acc: 0.123, kl: 1908744.503, kl_b: 1672.778|1573.845|47.135, kl_w: 1250718.713|638420.562|16311.473, ll: -971397.826, logpy: -17025.578, loss: 971397.826
MAP-Test 0 | Hq_b: -0.588|-0.582|-0.559, Hq_w: -0.581|-0.574|-0.582, acc: 0.123, kl: 2142486.000, kl_b: 1763.986|1078.362|49.512, kl_w: 1404631.000|716824.625|18138.572, ll: -1085891.787, logpy: -14648.787, loss: 1085891.787


  0%|          | 0/390 [00:00<?, ?it/s]

Train 1 | Hq_b: -0.588|-0.582|-0.557, Hq_w: -0.581|-0.574|-0.582, acc: 0.325, kl: 1900153.797, kl_b: 1605.559|1371.665|43.583, kl_w: 1247041.895|633751.246|16339.850, ll: -483825.665, logpy: -8787.215, loss: 483825.665
Test 1 | Hq_b: -0.588|-0.582|-0.555, Hq_w: -0.581|-0.574|-0.582, acc: 0.507, kl: 1891170.077, kl_b: 1574.637|1585.707|41.120, kl_w: 1242958.665|628845.148|16164.805, ll: -477513.093, logpy: -4720.575, loss: 477513.093
MAP-Test 1 | Hq_b: -0.588|-0.582|-0.555, Hq_w: -0.581|-0.574|-0.582, acc: 0.515, kl: 2125617.500, kl_b: 1678.274|1596.261|44.478, kl_w: 1396863.875|707341.312|18093.328, ll: -535921.405, logpy: -4517.032, loss: 535921.405


...

Train 47 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.584, acc: 0.973, kl: 1860152.108, kl_b: 1424.690|1186.656|32.835, kl_w: 1229199.056|612577.528|15731.344, ll: -65.518, logpy: -65.518, loss: 65.518
Test 47 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.584, acc: 0.941, kl: 1860205.798, kl_b: 1420.300|1256.430|32.010, kl_w: 1229146.976|612615.562|15734.511, ll: -273.447, logpy: -273.447, loss: 273.447
MAP-Test 47 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.584, acc: 0.950, kl: 2095077.250, kl_b: 1548.050|1694.633|35.937, kl_w: 1383039.500|691067.562|17691.568, ll: -232.559, logpy: -232.559, loss: 232.559


  0%|          | 0/390 [00:00<?, ?it/s]

Train 48 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.584, acc: 0.974, kl: 1860347.402, kl_b: 1430.255|1350.596|32.671, kl_w: 1229189.573|612613.567|15730.742, ll: -59.826, logpy: -59.826, loss: 59.826
Test 48 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.584, acc: 0.943, kl: 1860317.790, kl_b: 1453.179|1372.864|33.704, kl_w: 1229139.981|612587.651|15730.405, ll: -255.677, logpy: -255.677, loss: 255.677
MAP-Test 48 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.584, acc: 0.951, kl: 2095241.875, kl_b: 1620.884|1750.633|36.077, kl_w: 1383051.500|691092.938|17689.854, ll: -231.562, logpy: -231.562, loss: 231.562


100%|██████████| 78/78 [00:06<00:00, 12.41it/s]


Train 49 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.584, acc: 0.975, kl: 1860423.620, kl_b: 1423.501|1403.653|32.927, kl_w: 1229211.198|612623.000|15729.341, ll: -51.688, logpy: -51.688, loss: 51.688
Test 49 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.585, acc: 0.943, kl: 1860509.122, kl_b: 1440.766|1460.202|32.988, kl_w: 1229178.018|612669.321|15727.828, ll: -259.081, logpy: -259.081, loss: 259.081
MAP-Test 49 | Hq_b: -0.589|-0.582|-0.547, Hq_w: -0.581|-0.574|-0.585, acc: 0.952, kl: 2095312.375, kl_b: 1684.968|1725.372|36.120, kl_w: 1383063.375|691113.188|17689.283, ll: -230.659, logpy: -230.659, loss: 230.659


### Plotting results of $\beta$ experiments

In [ ]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib import rc
plt.style.use("seaborn")
import seaborn as sns
import re
import numpy as np
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=22)
plt.rcParams.update({'legend.fontsize': 16, 'xtick.labelsize': 16,
'ytick.labelsize': 16, 'axes.labelsize': 16})
def load_data(basename):
    files_no_transfer = [open("results/{0}_beta_{1}/diagnostics.txt".format(basename, beta_type)).read() for beta_type in ["Blundell", "Standard", "None"] ]
    
    accuracies = [list(map(lambda x: x.split(" ")[-1], re.findall(r"(\'acc\': \d.\d+)", file))) for file in files_no_transfer]
    valid = [acc[1::2] for acc in accuracies]
    return np.array(valid).astype(np.float32)

f = plt.figure(figsize=(10, 8))

current_palette = sns.color_palette()
vt = load_data("[0,1,2,3,4,5,6,7,8,9]")
x_ticks = range(vt.shape[1])
colors  = sns.color_palette(n_colors = 4)
legends  = [r"$\frac{2^{M-i}}{2^M-1}$", r"$\frac{1}{M}$", "0"]
for values, legend_name, color in zip(vt,legends, colors):
    plt.plot(x_ticks, values, label=r"Accuracy with transfer, with $\beta$= {0}".format(legend_name), color=color)

plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(x_ticks[::5], map(lambda x: x+1, x_ticks[::5]))
f.suptitle("Accuracy after training for 50 epochs")
plt.legend(loc = 7)

plt.savefig("figs/comparing_beta.pdf")

### Plotting results of Normalizing Flows experiments

In [1]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib import rc
plt.style.use("seaborn")
import seaborn as sns
import re
import numpy as np

plt.rc('font', family='serif', size=22)
plt.rcParams.update({'legend.fontsize': 16, 'xtick.labelsize': 16,
'ytick.labelsize': 16, 'axes.labelsize': 16})
def load_data(basename):
    normflow = open("norm_flow/{}.txt".format(basename)).read()
    
    accuracies = list(map(lambda x: x.split(" ")[-1], re.findall(r"(acc: \d.\d+)", normflow)))
    valid = accuracies[1::3]
    if basename=="beta_blundell":
        valid=valid[:50]
    return np.array(valid).astype(np.float32)

f = plt.figure(figsize=(10, 8))

current_palette = sns.color_palette()

colors = sns.color_palette(n_colors = 2)
basenames = ["beta_blundell", "logfile"]
legends = ["Without Normalizing Flows", "With Normalizing Flows"]
for basename, color, legend in zip(basenames, colors, legends):
    vt = load_data(basename)
    x_ticks = range(vt.shape[0])
    plt.plot(x_ticks, vt, label=legend, color=color)

plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(x_ticks[5::5], map(lambda x: x, x_ticks[5::5]))
f.suptitle("With and without Normalizing Flows")
plt.legend(loc = 7)

plt.savefig("figs/normflow.pdf")

### Plotting results of transfer learning experiments